In [1]:
import os
import sys
import json
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import udf, struct, count_distinct, from_unixtime
import multiprocessing

In [2]:
conf = SparkConf()
conf.set('spark.master', 'local[5]')
conf.set('spark.executor.memory', '2G')
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

22/11/17 13:47:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
file_path = '/home/jovyan/__AMEX/test_data.csv'
sdf = spark.read.csv(file_path, header=True)

In [4]:
sdf.limit(5).toPandas()

22/11/17 13:47:51 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
/opt/conda/lib/python3.9/site-packages/pyspark/sql/pandas/conversion.py:202: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df[column_name] = series


,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-02-19,0.631315348870127,0.0019117764764965,0.0107280865415314,0.8144968588558597,0.0075468996994418,0.1686508406762685,0.0099714525530773,0.0023469930565875,...,None,None,None,None,0.0046687390870528,None,None,None,0.0082807231606915,None
1,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-03-25,0.5870422600496985,0.0052751643361475,0.0110263694322631,0.8108478013948842,0.0018171296454829,0.2413889625766217,0.00016584141313932932,0.0091318432435674,...,None,None,None,0.0001420210291239854,0.004939899911698,0.0090213834243855,None,0.0036952732565555,0.003752937615896,0.0014599699636628
2,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-04-25,0.609056086399499,0.0033255515795944,0.0163899547111051,1.0046204377263357,0.00011385173627081912,0.2669761673635372,0.0041960837134838,0.0041915117538253,...,None,None,None,7.412888484163106e-05,0.0021140603899732,0.0046563052524218,None,0.0031548510598543,0.0021562450424162,0.0064823796768577
3,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-05-20,0.6149111732046417,0.0090646385392601,0.0216718794499841,0.8165485506797046,0.0097219768109759,0.1889469188595959,0.0041232573662345,0.0153252365586431,...,None,None,None,0.0047432604605391,0.0063924839314034,0.0028898648308047,None,0.0060443075576254,0.0052060027823573,0.0078546417568729
4,00000469ba478561f23a92a868bd366de6f6527a684c9a...,2019-06-15,0.5916728001946708,0.2387940614119742,0.0159229696665496,0.8104560849002189,0.0020262182221732,0.1800348589857536,0.0007314587099235914,0.0112806885679414,...,None,None,None,0.0081332381954749,0.0043291418077763,0.0083838696011743,None,0.0010084173537337,0.0074209562770936,0.009470804051453


In [5]:
sdf.count()

11363762